In [5]:
import os
import json
import numpy as np
import itertools


input_folder = "20node"
output_folder = "output_select_path_greedy"
os.makedirs(output_folder, exist_ok=True)

# tính ma trận thời gian di chuyển 
def compute_travel_time(cluster_heads, positions, v_AUV=10.0):
    n = len(cluster_heads)
    travel_time = np.zeros((n, n))
    for i in range(n):
        for j in range(n):
            if i != j:
                dist = np.linalg.norm(positions[cluster_heads[i]] - positions[cluster_heads[j]])
                travel_time[i][j] = dist / v_AUV
    return travel_time

# chọn đường đi 
def path_selection(cluster_heads, travel_time, positions, v_AUV):
    n = len(cluster_heads)
    best_time = float('inf')
    best_path = None
    O = np.array([0, 0, 0])
    for path in itertools.permutations(range(n)):
        total_time = 0
        start_head_cluster = cluster_heads[path[0]]
        total_time += np.linalg.norm(positions[start_head_cluster] - O) / v_AUV
        total_time += sum(travel_time[path[i-1]][path[i]] for i in range(1, n))
        end_head = cluster_heads[path[-1]]
        total_time += np.linalg.norm(positions[end_head] - O) / v_AUV
        if total_time < best_time:
            best_time = total_time
            best_path = path
    return best_path, best_time


for filename in os.listdir(input_folder):
    if not filename.endswith(".json"):
        continue

    input_path = os.path.join(input_folder, filename)
    output_path = os.path.join(output_folder, f"result_{filename}")

    print(f"\nĐang xử lý {filename}")

    with open(input_path, "r", encoding="utf-8") as f:
        nodes_data = json.load(f)

    # Trích danh sách cluster head và tọa độ tâm cụm
    cluster_heads = []
    positions = {}
    for cid, info in nodes_data.items():
        ch = info["cluster_head"]
        cluster_heads.append(ch)
        positions[ch] = np.array(info["center"])

    # Tính ma trận thời gian di chuyển
    travel_time = compute_travel_time(cluster_heads, positions)

    # Tìm đường đi tối ưu
    best_path, best_time = path_selection(cluster_heads, travel_time, positions, v_AUV=10.0)

    # Xuất kết quả
    result = {
        "input_file": filename,
        "cluster_heads": cluster_heads,
        "best_path": [cluster_heads[i] for i in best_path],
        "total_time": round(float(best_time), 4)
    }

    with open(output_path, "w", encoding="utf-8") as f_out:
        json.dump(result, f_out, ensure_ascii=False, indent=4)

    print(f"Kết quả của {output_path}:")
    print(f"Tổng thời gian di chuyển nhỏ nhất: {best_time:.2f} s")

print("\nDone")



Đang xử lý nodes_20.json
Kết quả của output_select_path_greedy\result_nodes_20.json:
Tổng thời gian di chuyển nhỏ nhất: 41.99 s

Done


In [6]:
def simulate_auv_cycle(data, r_sen=100, R=20, E_tx=5, E_rx=2, v_AUV=10, death_threshold=0.9):
    """
    Mô phỏng quá trình AUV thu thập dữ liệu nhiều chu kỳ.
    - Mạng dừng khi tỷ lệ node chết > death_threshold (m/n > 0.9)
    """
    positions = np.array([[d["x"], d["y"], d["z"]] for d in data])
    energies = {d["id"]: d["energy_node"] for d in data}
    node_ids = [d["id"] for d in data]
    total_nodes = len(node_ids)  # 🟩 tổng số node ban đầu
    cycle = 0
    total_travel_time = 0.0

    while True:
        alive_ids = [nid for nid, e in energies.items() if e > 0]
        alive_count = len(alive_ids)
        dead_count = total_nodes - alive_count
        dead_ratio = dead_count / total_nodes   # 🟩 Tính tỷ lệ m/n

        # 🟩 Kiểm tra điều kiện dừng theo tỷ lệ node chết
        if dead_ratio >= death_threshold:
            print(f"\n⚠️ Mạng sập: {dead_count}/{total_nodes} nodes chết ({dead_ratio*100:.1f}%)")
            break

        cycle += 1
        print(f"\n🔁 Chu kỳ {cycle}: {alive_count}/{total_nodes} nodes hoạt động")

        alive_positions = np.array([positions[nid] for nid in alive_ids])

        # Phân cụm các node còn sống
        clusters_raw = cluster_split(alive_positions, alive_ids, r_sen=r_sen, R=R)
        cluster_heads = []
        positions_dict = {}

        for c in clusters_raw:
            ch = choose_cluster_head(c, energies)
            cluster_heads.append(ch)
            positions_dict[ch] = np.mean(c["nodes"], axis=0)

        # Tính ma trận thời gian di chuyển và đường đi tối ưu
        travel_time = compute_travel_time(cluster_heads, positions_dict, v_AUV)
        best_path, best_time = path_selection(cluster_heads, travel_time)

        # Tính thêm thời gian từ O đi và về
        O = np.array([0, 0, 0])
        dist_start = np.linalg.norm(O - positions_dict[cluster_heads[best_path[0]]])
        dist_end = np.linalg.norm(O - positions_dict[cluster_heads[best_path[-1]]])
        cycle_time = best_time + (dist_start + dist_end) / v_AUV
        total_travel_time += cycle_time

        # Cập nhật năng lượng các node
        for c in clusters_raw:
            ch = choose_cluster_head(c, energies)
            energies[ch] -= E_tx
            for nid in c["node_ids"]:
                if nid != ch:
                    energies[nid] -= E_rx
                    if energies[nid] < 0:
                        energies[nid] = 0

        print(f"⏱ Thời gian chu kỳ {cycle}: {cycle_time:.2f}s | "
              f"Tỷ lệ node chết: {dead_ratio*100:.1f}%")

    print(f"\n✅ Tổng số chu kỳ: {cycle}")
    print(f"⏱ Tổng thời gian di chuyển của AUV: {total_travel_time:.2f}s")
    return cycle, total_travel_time
